# Steenrod barcode of a filtration of $\mathbb RP^2$

In this notebook we use ``steenroder`` to computing the Steenrod barcode of a small filtration.
The filtered complex $X$ that we consider is a model for the real projective plane

<img src="filtered_rp2.png" width="200" height="200">

represented by the following `tuple` of `tuple` of `int`:

In [ ]:
rp2 = (
    (0,), 
    (1,), (0,1), 
    (2,), (0,2), (1,2), (0,1,2),
    (3,), (0,3), (1,3), (0,1,3), (2,3),
    (4,), (0,4), (1,4), (2,4), (1,2,4), (3,4), (0,3,4), (2,3,4),
    (5,), (0,5), (1,5), (2,5), (0,2,5), (3,5), (1,3,5), (2,3,5), (4,5), (0,4,5), (1,4,5)
    )

filtration = rp2
max_dim = 2

## What will be computed

### Relative cohomology

We will consider the persistent relative cohomology of $X$

$$
H^\bullet(X) \leftarrow H^\bullet(X, X_{0}) \leftarrow H^\bullet(X, X_{1}) \leftarrow \cdots \leftarrow H^\bullet(X, X_{m})
$$
as a persistence module.

### Regular barcode

For $i < j \in \{-1,\dots,m\}$ let $X_{ij}$ be the unique composition $H^\bullet(X, X_{i}) \leftarrow H^\bullet(X, X_{j})$ with the convention $X_{-1} = \emptyset$.
The barcode of this persistence module is the multiset

$$
Bar_X =
\big\{ [p,q] \mid -1 \le p < q \le m \big\}
$$

defined by the property

$$
\mathrm{rank} \, X_{ij} =
\mathrm{card} \big\{[p,q] \in Bar_X \mid p \le i < j \le q \big\}.
$$

### Steenrod barcode

For every cohomology operation, say $Sq^k$, we obtain an endomorphism of persistent relative cohomology

\begin{align*}
&H^\bullet(X) \leftarrow \cdots \leftarrow H^\bullet(X, X_{m}) \\
& {\tiny Sq^k} \uparrow \kern 2.5cm {\tiny Sq^k} \uparrow\\
&H^\bullet(X) \leftarrow \cdots \leftarrow H^\bullet(X, X_{m}).
\end{align*}

The $Sq^k$-barcode of $X$ is the barcode of the image of this endomorphism.

In [ ]:
from steenroder import *

k = 1
d = 2
barcode, st_barcode = barcodes(k, filtration)

for dd in range(d+1):
    print(f'Regular barcode in dim {dd}: \n{barcode[dd]}')
for dd in range(d):
    print(f'Sq^{k}-barcode in dim {dd+k}: \n{st_barcode[dd+k]}')

There are three infinite bars in the regular barcode:

[-1,0] in deg 0,

[-1,11] in deg 1, and

[-1,30] in deg 2.

Additionally, there is one $Sq^1$-bar:

[-1,11] in deg 2.

This $Sq^1$-bar witnesses in the persistent context the non-trivial relationship $Sq^1 [\alpha_1] = [\alpha_2]$ between the degree 1  and 2 generators of the cohomology of $\mathbb R P^2$.

## How to compute these invariants

### Regular barcode

#### Boundary matrix

Order the simplices $a_0 < a_1 < \dots < a_m$ of $X$ so that $X_k = \{a_i \mid i \leq k\}$.
Consider the matrix $D$ representing the boundary map $\partial \colon C_\bullet(X;\mathbb{F}_2) \to C_{\bullet-1}(X;\mathbb{F}_2)$ with respect to the (ordered) basis determined by the (ordered) simplices.
Explicitly,

$$
\partial a_j = \sum_{i=0}^m D_{ij} \, a_i.
$$

Notice that the submatrix $D_{\leq k \leq k}$ represents the boundary of $C_\bullet(X_k;\mathbb F_2)$ for every $k \in \{0,\dots,m\}$.

#### Anti-transpose

We will consider the antitranspose $D^\perp$ of the boundary matrix $D$.
It is defined for every $p,q \in \{0,\dots,m\}$ by

$$
D^\perp_{p,\, q} = D_{m-q,\ m-p}.
$$

For any $k \in \{0,\dots,m\}$ the submatrix $D^\perp_{\leq k, \leq k}$ represents the coboundary map $\delta \colon C^\bullet(X, X_{m-k-1}; \mathbb{F}_2) \to C^{\bullet+1}(X, X_{m-k-1}; \mathbb{F}_2)$ with respect to the (ordered) dual basis $a_m^\bullet < \dots < a_{m-k}^\bullet$.


#### The $R = D^\perp V$ decomposition

Here $V$ is an invertible upper triangular matrix and $R$ is reduced, i.e., no two columns have the same pivot row.

Denoting the $i$-th column of $R$ by $R_{i}$ where $i \in \{0,\dots,m\}$, let

$$
P = \{ i \ |\ R_i = 0\}, \qquad N = \{ i \ |\ R_i \neq 0\}, \qquad E = P \setminus \{\text{pivots of } R\}.
$$

#### The regular barcode

There exists a canonical bijection between the union of $N$ and $E$ and the persistence relative cohomology barcode of the filtration given by

\begin{align*}
N \ni j &\mapsto \big[ (m-j), (m-\mathrm{pivot}\,R_j) \big] \in Bar^{\dim(j)+1}_X \\
E \ni j &\mapsto \big[ -1, (m-j) \big] \in Bar^{\dim(j)}_X
\end{align*}

#### Representatives

Additionally, a representative cocycle is given by
\begin{equation*}
[i,j] \mapsto
\begin{cases}
V_{m-j}, & i = -1, \\
R_{m-i}, & i \neq -1.
\end{cases}
\end{equation*}

and a basis of coboundaries in $C^\bullet(X, X_{m-j-1})$ corresponds to non-zero vectors in

\begin{equation*}
\{R_i\ |\ i \in N,\, i \leq j\}.
\end{equation*}

### Regular barcode using ``steenroder``

We will illustrate the ``steenroder`` implementation of the above algorithm using a small filtration of the circle.

<img src="filtered_circle.png" width="200" height="200">

In [ ]:
# circle = (
#     (0,), 
#     (1,), (0,1), 
#     (2,), (1,2), (0,2)
#     )
# filtration = circle
# max_dim = 1

#### Splitting by dimension

In `steenroder` all computations are done dimension by dimension. We start by splitting the filtration into collection of simplices of the same dimension.

In [ ]:
filtration_by_dim = sort_filtration_by_dim(filtration)

The output of ``sort_filtration_by_dim`` is as follows:

For each dimension ``d``, a list of 2 aligned int arrays: the first is a 1D array containing the (ordered) positional indices of all ``d``-dimensional simplices in `filtration`; the second is a 2D array whose ``i``-th row is the (sorted) collection of vertices defining the ``i``-th ``d``-dimensional simplex.

In [ ]:
d = 1
filtration_by_dim[d]

#### A note on anti-transposition

The anti-transposition operation $(-)^\perp$ on a matrix $N$ is determined by composing the usual trnasposition

$$
(-)^\mathrm{T} \colon (i,j) \mapsto (j, i),
$$
and the horizontal and vertical flips
$$
(-)^\mathrm{A} \colon (i,j) \mapsto (p-i, q-j)
$$

where we indexed rows and columns from $0$ to $p$ and $q$ respectively.

We remark that

$$
(M N)^\mathrm{A} = M^\mathrm{A} N^\mathrm{A}.
$$

for any pair of square matrices $M$ and $N$.

#### The $R = D^\perp V$ decomposition

To compute the matrices $R$ and $V$ in the decomposition $R = D^\perp V$ ``steenroder`` uses the method ``get_reduced_triangular``.
It actually produces $R^\mathrm{A}$ and $V^\mathrm{A}$.
More precisely, the output ``reduced`` is a tuple of ``numba.typed.List`` with ``reduced[d]`` the ``d``-dimensional part of $R^\mathrm{A}$.
More explicitly, for an integer $j$ we have that an integer $i$ is in the tuple ``reduced[d][j]`` if and only if $R^\mathrm{A}_{ij} = 1$. 
Similarly ``triangular[d]`` is the ``d``-dimensional part of the $V^\mathrm{A}$. There are also two other auxiliary outputs that will be discussed later, these are ``spx2idx`` and ``idxs``.

In [ ]:
spx2idx, idxs, reduced, triangular = get_reduced_triangular(filtration_by_dim)

We will verify for our running example that $R = D^\perp V$ by checking that the coboundary $D^\mathrm{T}$ of this filtration is equal to $R^\mathrm{A} U^\mathrm{A}$ where $U$ is the inverse of $V$.

In [ ]:
m = len(filtration)-1

# compute coboundary
coboundary = np.zeros((m+1,m+1), dtype=int)
for j, x in enumerate(filtration):
    for i, y in enumerate(filtration):
        if len(x)+1 == len(y) and set(x).issubset(set(y)):   
            coboundary[i,j] = 1
    
def to_array(matrix, e=0):
    """Transforms reduced (e=1) and triangular (e=0) to numpy arrays"""
    array = np.zeros((m+1,m+1), dtype=int)
    for d in range(max_dim+1):
        for i, col in enumerate(matrix[d]):
            for j in col:
                array[idxs[d+e][j], idxs[d][i]] = 1
    return array

antiR, antiV = to_array(reduced, e=1), to_array(triangular, e=0)
antiU = np.array(np.linalg.inv(antiV), dtype=int)
product = np.matmul(antiR, antiU) % 2
print(f'D^T = R^AU^A: {(coboundary == product).all()}')

#### Regular barcode and representatives

In [ ]:
barcode, coho_reps = get_barcode_and_coho_reps(idxs, reduced, triangular)

For every ``d`` we have that ``barcode[d]`` is a 2D int array of shape ``(n_bars, 2)`` containing the birth (entry 1) and death (entry 0) indices of persistent relative cohomology classes in degree ``d``.

In [ ]:
for d in range(max_dim+1):
    print(f'The barcode in dim {d}:\n{barcode[d]}')

We can compare to the description
\begin{align*}
N \ni j &\mapsto \big[ (m-j), (m-\mathrm{pivot}\,R_j) \big] \in Bar_{{\dim(j)+1}}^{\mathrm{fin}} \\
E \ni j &\mapsto \big[ -1, (m-j) \big] \in Bar_{\dim(j)}^{\mathrm{inf}}
\end{align*}
where
$$
P = \{ i \ |\ R_i = 0\}, \qquad N = \{ i \ |\ R_i \neq 0\}, \qquad E = P \setminus \{\text{pivots of } R\}.
$$

In [ ]:
R = np.flip(antiR, [0,1])
k = m+1  # for the whole matrix use m+1
# print(f'R = \n{R[:k,:k]}')

In [ ]:
def pivot(column):
    try:
        return max(column.nonzero()[0])
    except:
        pass

P, N, E = [], [], []
for i, col in enumerate(R.T):
    if pivot(col):
        N.append(i)
        E.remove(pivot(col))
    else:
        P.append(i)
        E.append(i)

print(f'P = {P}\nN = {N}\nE = {E}')

In [ ]:
bcode = [[] for d in range(max_dim+1)]

for j in N:
    d = len(filtration[m-j])
    bcode[d].append([m-j, m-pivot(R[:,j])])
        
for j in E:
    d = len(filtration[m-j])-1
    bcode[d].append([-1, m-j])

for d in range(max_dim+1):
    print(f'The barcode in dim {d}:\n{bcode[d]}')

Additionally, ``coho_reps[d][k]`` is a list of positional indices, relative to the ``d``-dimensional portion of the filtration, representing the bar ``barcode[d][k]`` for every ``k``.

To express these representatives in terms of cochains, we use that ``idxs[d]`` is an int array containing the (ordered) positional indices of all ``d``-dimensional simplices in the filtration.

In [ ]:
d = 1
print(f'In dim {d}:')
for bar, rel_positions in zip(barcode[d], coho_reps[d]):
    coho_rep = []
    for p in rel_positions:
        coho_rep.append(filtration[idxs[d][p]])
    print(f"{str(bar): <7} rep. by {coho_rep}")

We can compare to the description
\begin{equation*}
[i,j] \mapsto
\begin{cases}
V_{m-j}, & i = -1, \\
R_{m-i}, & i \neq -1.
\end{cases}
\end{equation*}

In [ ]:
V = np.flip(antiV, [0,1])
d = 1
print(f'In dim {d}:')
for bar in barcode[d]:
    i,j = bar
    c_rep = []
    if i == -1:
        nonzeros = np.nonzero(V[:,m-j])[0]
    else:
        nonzeros = np.nonzero(R[:,m-i])[0]
    for p in nonzeros:
        c_rep.append(filtration[m-p])
    print(f'{str(bar): <7} rep. by {c_rep}')

### Steenrod barcodes

We will now describe an effective computation of the $Sq^k$-barcode of persistent relative cohomology bulding on the computation of regular barcodes and representatives.
Recall that the $Sq^k$-barcode is by definition the barcode of the image persistent module of the endomorphism $Sq^k$.


#### Steenrod representatives

Given vector $v$ corresponding to a cocycle $\alpha$, let $SQ^k(v)$ be the vector correponding to cocycle representative of $Sq^k \big( [\alpha] \big)$.
For example, the one obtained using the following pseudo-code, where $A$ is the support of $\alpha$.

<img src="stsq.png" width="550" height="550">

#### Steenrod matrix

Identifying a vector with the support of its associated cochain, define $Q^k$ to be the matrix whose columns are given by

\begin{equation*}
Q^k_i = \begin{cases}
SQ^k(V_i) & i \in E, \\
SQ^k(R_j) & i = pivot(R_j), \\
0 & otherwise.
\end{cases}
\end{equation*}

#### Reduction

Given $R$ and $Q^k$, we denote by $R_{\le j}$ and $Q^k_{\le j}$ the submatrices containing all columns with indices less than or equal to $j$, and $R_{\le j} \mid Q^k_{\le j}$ the matrix obtained by concatenating their columns.
With this notation the following pseudo-code computes the $k$-Steenrod barcode of the filtration.

TYPO

<img src="st_bars.png" width="600" height="600">

## Steenrod barcodes using ``steenroder``

Let us return to the $\mathbb R P^2$ example.

In [ ]:
filtration = rp2
filtration_by_dim = sort_filtration_by_dim(filtration)
spx2idx, idxs, reduced, triangular = get_reduced_triangular(filtration_by_dim)
barcode, coho_reps = get_barcode_and_coho_reps(idxs, reduced, triangular)

Using a fast implementation of Algorithm 2, ``steenroder`` constructs the Steenrod matrix using the method ``get_steenrod_matrix``.
Its output is a list of ``numba.typed.List``, one list per simplex dimension.
Explicitly, ``steenrod_matrix[d][j]`` entry is the result of computing the Steenrod square of ``coho_reps[d][j]``.

In [ ]:
k = 1
steenrod_matrix = get_steenrod_matrix(k, coho_reps, filtration_by_dim, spx2idx, n_jobs=-1)

d = 1
print(f'In dim {d}:')
for bar, rel_pos, st_rel_pos in zip(barcode[d], coho_reps[d], steenrod_matrix[d+1]):
    coho_rep = []
    for p in rel_pos:
        coho_rep.append(filtration[idxs[d][p]])
    st_rep = []
    for p in st_rel_pos:
        st_rep.append(filtration[idxs[d+1][p]])
    print(f'bar {bar} rep. by {coho_rep}')
    print(f'whose sq^{k} img. is rep. by {st_rep}\n')

Algorithm 2 is carried through in ``get_steenrod_barcode``, whose output is the $Sq^k$-barcode of the filtration.

In [ ]:
d = 2
st_bcode = get_steenrod_barcode(k, steenrod_matrix, idxs, reduced, barcode, filtration_values=None)
st_bcode[2]

In [ ]:
len(steenrod_matrix[2])
len(idxs[1])
barcode[1]

In [ ]:
k=1
antiQ = np.zeros((m+1,m+1), dtype=int)
for d in range(max_dim-k+1):
    for bar, col in zip(barcode[d], steenrod_matrix[d+k]):
        for p in col:
            antiQ[idxs[d+k][p], m-bar[1]] = 1

Q = np.flip(antiQ, [0,1])

In [ ]:
def _pivot(column):
    try:
        return max(column.nonzero()[0])
    except ValueError:
        return None

def reduce_vector(reduced, vector):
    num_col = reduced.shape[1]
    i = -1
    while i >= -num_col:
        if not np.any(vector):
            break
        else:
            piv_v = _pivot(vector)
            piv_i = _pivot(reduced[:, i])

            if piv_i == piv_v:
                vector[:, 0] = np.logical_xor(reduced[:, i], vector[:, 0])
                i = 0
            i -= 1

def reduce_matrix(reduced, matrix):
    num_vector = matrix.shape[1]
    reducing = reduced.copy()

    for i in range(num_vector):
        reduce_vector(reducing, matrix[:, i:i + 1])
        reducing = np.concatenate([reducing, matrix[:, i:i + 1]], axis=1)

def get_steenrod_barcode(reduced, steenrod_matrix):
    num_col = reduced.shape[1]
    alive = {i: True for i in range(num_col)}
    R = reduced
    Q = steenrod_matrix
    st_barcode = []
    for j in range(num_col):
        reduce_matrix(R[:, :j+1], Q[:, :j+1])
        for i in range(j+1):
            if alive[i] and not np.any(Q[:, i]):
                alive[i] = False
                if j > i:
                    st_barcode.append([i, j])
    st_barcode += [[-1,i] for i in alive if alive[i]]
    return sorted([bar for bar in st_barcode if bar[1] > bar[0]])

In [ ]:
get_steenrod_barcode(R,Q)